In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [7]:
df.shape

(50000, 2)

In [8]:
df["review"][3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [9]:
import seaborn as sns

sns.countplot(x='sentiment', data=df)

In [10]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [11]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)


In [12]:
X = []
sentences = list(df['review'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [13]:
X[3]

'Basically there a family where little boy Jake thinks there a zombie in his closet his parents are fighting all the time This movie is slower than soap opera and suddenly Jake decides to become Rambo and kill the zombie OK first of all when you re going to make film you must Decide if its thriller or drama As drama the movie is watchable Parents are divorcing arguing like in real life And then we have Jake with his closet which totally ruins all the film expected to see BOOGEYMAN similar movie and instead watched drama with some meaningless thriller spots out of just for the well playing parents descent dialogs As for the shots with Jake just ignore them '

In [14]:
type(X)

list

In [15]:
df1 = pd.Series((v[0] for v in X))


In [16]:
df2 = pd.Series(X)

In [17]:
df2[3]

'Basically there a family where little boy Jake thinks there a zombie in his closet his parents are fighting all the time This movie is slower than soap opera and suddenly Jake decides to become Rambo and kill the zombie OK first of all when you re going to make film you must Decide if its thriller or drama As drama the movie is watchable Parents are divorcing arguing like in real life And then we have Jake with his closet which totally ruins all the film expected to see BOOGEYMAN similar movie and instead watched drama with some meaningless thriller spots out of just for the well playing parents descent dialogs As for the shots with Jake just ignore them '

In [18]:
X = df2

In [19]:
y = df['sentiment']

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [22]:
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(33500, 85783)

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(33500, 85783)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(33500, 85783)

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC


In [26]:
svc_model = LinearSVC()
svc_model.fit(X_train_tfidf,y_train)
# predictions = svc_model.predict(X_test)
# print(metrics.classification_report(y_test,predictions))
# print(metrics.accuracy_score(y_test,predictions))

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [27]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [28]:
predictions = text_clf.predict(X_test)

In [29]:
from sklearn import metrics


print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

    negative       0.91      0.89      0.90      8208
    positive       0.89      0.91      0.90      8292

   micro avg       0.90      0.90      0.90     16500
   macro avg       0.90      0.90      0.90     16500
weighted avg       0.90      0.90      0.90     16500



In [32]:
def func1(dataset):
    df = pd.read_csv(dataset)
    df.drop(['web-scraper-order', 'web-scraper-start-url','captions'], axis=1,inplace=True)
    positive = []
    negative = []
    i = 0
    for i in range(0,10):
        
        #print("i is ",i)
        test = df.sample(n=50)
        for j in range(0,50):
            test2 = test.iloc[j]
            res = text_clf.predict(test2)
            #print(res)
            if res == 'positive':
                positive.append(res)
            elif res == 'negative':
                negative.append(res)
    #print(type(positive), " po is ",positive)
    #print(type(negative)," negative is ",negative)
    print("total positive reviews are ",len(positive))
    print("total negative reviews are ",len(negative))
    print("% of positive reviews are ", (len(positive)*2)/10)
    print("% of negative reviews are ", (len(negative)*2)/10)

        
            
        
        

In [37]:
func1('american assassin.csv')#508 both

total positive reviews are  192
total negative reviews are  308
% of positive reviews are  38.4
% of negative reviews are  61.6


In [40]:
func1('interstellar.csv')#imdb 5156 df 3426

total positive reviews are  348
total negative reviews are  152
% of positive reviews are  69.6
% of negative reviews are  30.4


In [84]:
func1('the_man_from_uncle.csv') #imdb 577 df 436

total positive reviews are  327
total negative reviews are  173
% of positive reviews are  65.4
% of negative reviews are  34.6


In [47]:
func1('mission impossible fallout.csv') #imdb 1747 df 1747

total positive reviews are  342
total negative reviews are  158
% of positive reviews are  68.4
% of negative reviews are  31.6


In [53]:
func1('predestination.csv')#imdb 737 df 737

total positive reviews are  371
total negative reviews are  129
% of positive reviews are  74.2
% of negative reviews are  25.8


In [58]:
func1('the 355.csv') #imdb 440 df 437

total positive reviews are  231
total negative reviews are  269
% of positive reviews are  46.2
% of negative reviews are  53.8


In [64]:
# rankings:tenet
# 1]predestination
# 2]interstellar
# 3]mission impossible fallout
# 4]man from uncle
# 5]the 355
# 5]american assassin

3425


In [73]:
def func2(dataset):
    df = pd.read_csv(dataset)
    df.drop(['web-scraper-order', 'web-scraper-start-url','captions'], axis=1,inplace=True)
    length = df.shape[0]
    positive = []
    negative = []
    for i in range(0,length):
        test2 = df.iloc[i]
        res = text_clf.predict(test2)
        #print(res)
        if res == 'positive':
            positive.append(res)
        elif res == 'negative':
            negative.append(res)
    #print(type(positive), " po is ",positive)
    #print(type(negative)," negative is ",negative)
    print("total length is ",length)
    print("total positive reviews are ",len(positive))
    print("total negative reviews are ",len(negative))
    print("% of positive reviews are ", (len(positive)/length)*100)
    print("% of negative reviews are ", (len(negative)/length)*100)

        
    

In [74]:
func2('american assassin.csv')

total length is  507
total positive reviews are  178
total negative reviews are  329
% of positive reviews are  35.108481262327416
% of negative reviews are  64.89151873767258


In [75]:
func2('interstellar.csv')

total length is  3425
total positive reviews are  2316
total negative reviews are  1109
% of positive reviews are  67.62043795620438
% of negative reviews are  32.379562043795616


In [86]:
func2('the_man_from_uncle.csv')

total length is  577
total positive reviews are  373
total negative reviews are  204
% of positive reviews are  64.64471403812824
% of negative reviews are  35.35528596187175


In [77]:
func2('mission impossible fallout.csv')

total length is  1746
total positive reviews are  1146
total negative reviews are  600
% of positive reviews are  65.63573883161511
% of negative reviews are  34.36426116838488


In [78]:
func2('predestination.csv')

total length is  736
total positive reviews are  520
total negative reviews are  216
% of positive reviews are  70.65217391304348
% of negative reviews are  29.347826086956523


In [79]:
func2('the 355.csv')

total length is  436
total positive reviews are  197
total negative reviews are  239
% of positive reviews are  45.18348623853211
% of negative reviews are  54.81651376146789


In [ ]:
# #ranking on total
# 1]predestination
# 2]interstellar
# 3]mission impossible fallout
# 4]the man from uncle
# 5]the 355
# 6] american assassin